#12.1 Your First Agent

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def multiply(a, b):
    return a * b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=multiply,
            name="Multiplication Calculator",
            description="Use this to perform multiplications of two numbers. This tool take two arguments, both  should be numbers.",
        ),
    ],
)

prompt = "Cost of $2.4 x $45.3 x $23.7"
# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt)
# agent.invoke(prompt)

AIMessage(content='To find the cost of $2.4 multiplied by $45.3 multiplied by $23.7, you would simply multiply the three numbers together:\n\n$2.4 x $45.3 x $23.7 = $2.4 x 45.3 x 23.7 = $258.48 x 23.7 = $6130.776\n\nTherefore, the cost of $2.4 multiplied by $45.3 multiplied by $23.7 is $6130.776.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 23, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c851ab21-b463-4b5c-83d5-d7a2b7451c16-0')

In [3]:
agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought:
To find the cost of $2.4 multiplied by $45.3 and then multiplied by $23.7, I can use the Multiplication Calculator tool.

Action:
```
{
  "action": "Multiplication Calculator",
  "action_input": {
    "a": 2.4,
    "b": 45.3
  }
}
```
Observation: 108.71999999999998
Thought:Now that we have the result of the first multiplication, we can proceed to multiply it by $23.7 to find the final cost.
Action: 
```
{
  "action": "Multiplication Calculator",
  "action_input": {
    "a": 108.72,
    "b": 23.7
  }
}
```
Observation: 2576.6639999999998
Thought:Now that we have the final result of multiplying $2.4 by $45.3 and then by $23.7, we can provide the total cost.
Action:
```
{
  "action": "Final Answer",
  "action_input": "The cost is $2576.66"
}
```

> Finished chain.


{'input': 'Cost of $2.4 x $45.3 x $23.7', 'output': 'The cost is $2576.66'}

#12.3 Zero-shot ReAct Agent

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def multiply(inputs):
    a, b = inputs.split(",")
    return float(a) * float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=multiply,
            name="Multiply Calculator",
            description="Use this to perform multiplications of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample:1,2",
        ),
    ],
)

prompt = "Cost of $2.4 x $45.3 x $23.7"
# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

/var/folders/jm/6pbq_zts6k12d5j3w0c7pr6h0000gn/T/ipykernel_12550/3946161441.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.1)
/var/folders/jm/6pbq_zts6k12d5j3w0c7pr6h0000gn/T/ipykernel_12550/3946161441.py:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from



> Entering new AgentExecutor chain...
I need to multiply these three numbers together to find the total cost.
Action: Multiply Calculator
Action Input: 2.4,45.3
Observation: 108.71999999999998
Thought:I need to multiply the result by the third number.
Action: Multiply Calculator
Action Input: 108.72,23.7
Observation: 2576.6639999999998
Thought:I now know the final answer
Final Answer: $2576.66

> Finished chain.


{'input': 'Cost of $2.4 x $45.3 x $23.7', 'output': '$2576.66'}

#12.4 OpenAI Functions Agent

In [2]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


class CalculatorTool(BaseTool):
    name: str = "CalculatorTool"
    description: str = """
    Use this to perform multiplications of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a * b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Cost of $2.4 x $45.3 x $23.7"
# prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

/var/folders/jm/6pbq_zts6k12d5j3w0c7pr6h0000gn/T/ipykernel_14170/3833385283.py:28: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 2.4, 'b': 45.3}`


108.71999999999998
Invoking: `CalculatorTool` with `{'a': 108.72, 'b': 23.7}`


2576.6639999999998The cost of $2.4 x $45.3 x $23.7 is $2576.66.

> Finished chain.


{'input': 'Cost of $2.4 x $45.3 x $23.7',
 'output': 'The cost of $2.4 x $45.3 x $23.7 is $2576.66.'}

#12.5 Search Tool

In [10]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools import DuckDuckGoSearchResults

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name: str = "StockMarketSymbolSearchTool"
    description: str = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Tesla's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...


/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Tesla'}`


snippet: 4、在连锁零售门店中有时称单品为一个SKU（中文译为最小存货单位，英文全称为 stock keeping unit, 简称SKU，定义为保存库存控制的最小可用单位，例如纺织品中一个SKU通常表示规格， …, title: SKU到底是什么意思？大白话说一下，最好举个例子，谢谢 - 知乎, link: https://www.zhihu.com/question/292246379, snippet: 先看看stock一词各个义项在英语中的使用分部，可见 “股票” 必然是绝大多数时候的含义，而它的本意 “树干” 早就很少使用了，这中间的过程，我们结合时间线和衍生脉络一起分析： stock 一 …, title: 怎么记 stock 这个单词的词意？ - 知乎, link: https://www.zhihu.com/question/448182657, snippet: Aug 19, 2021 · 金融大佬可以帮忙解释翻译一下stock, security, fund, bond这几个词吗？ 原文是讲中国改革开放和市场改革的。 句子：China is boldly moving beyond stocks into new types …, title: 金融大佬可以帮忙解释翻译一下stock, security, fund, bond这几个 …, link: https://www.zhihu.com/question/480942889, snippet: 投资炒股常用的10个APP，你最钟意哪一款？ 每个券商都有自己的交易APP，券商自己的APP一般只支持自家券商账户登录交易，部分券商也支持三方平台进行登录交易，因为三方平台可以支 …, title: 投资炒股常用的10个APP，你最钟意哪一款？ - 知乎, link: https://www.zhihu.com/question/401767471The stock market symbol for Tesla is TSLA. Now, let's gather information on Tesla

/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


{'input': "Give me information on Tesla's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "The stock market symbol for Tesla is TSLA. Now, let's gather information on Tesla's stock performance and analyze if it's a potential good investment.\nTo analyze if Tesla's stock is a potential good investment, we need to consider various factors such as:\n\n1. **Financial Performance**: Look at Tesla's revenue growth, profitability, and cash flow. Analyze if the company is consistently growing and generating profits.\n\n2. **Market Trends**: Consider the trends in the electric vehicle industry and how Tesla is positioned compared to its competitors.\n\n3. **Future Growth Potential**: Evaluate Tesla's future growth prospects, including new product launches, expansion into new markets, and technological advancements.\n\n4. **Risk Factors**: Assess any risks associated with investing in Tesla, such as regulatory challenges, 

#12.6 Stock Information Tools

In [15]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools import DuckDuckGoSearchResults

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name: str = "StockMarketSymbolSearchTool"
    description: str = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name: str = "CompanyOverview"
    description: str = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name: str = "CompanyIncomeStatement"
    description: str = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name: str = "CompanyStockPerformance"
    description: str = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Tesla's stock, considering its financials, income statements help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...


/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Tesla'}`


snippet: 4、在连锁零售门店中有时称单品为一个SKU（中文译为最小存货单位，英文全称为 stock keeping unit, 简称SKU，定义为保存 …, title: SKU到底是什么意思？大白话说一下，最好举个例子，谢谢 - 知乎, link: https://www.zhihu.com/question/292246379, snippet: 先看看stock一词各个义项在英语中的使用分部，可见 “股票” 必然是绝大多数时候的含义，而它的本意 “树干” 早就很少使用了，这中间 …, title: 怎么记 stock 这个单词的词意？ - 知乎, link: https://www.zhihu.com/question/448182657, snippet: Aug 19, 2021 · 金融大佬可以帮忙解释翻译一下stock, security, fund, bond这几个词吗？ 原文是讲中国改革开放和市场改革的。 句 …, title: 金融大佬可以帮忙解释翻译一下stock, security, fund, bond这几个 …, link: https://www.zhihu.com/question/480942889, snippet: 投资炒股常用的10个APP，你最钟意哪一款？ 每个券商都有自己的交易APP，券商自己的APP一般只支持自家券商账户登录交易， …, title: 投资炒股常用的10个APP，你最钟意哪一款？ - 知乎, link: https://www.zhihu.com/question/401767471

/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()



Invoking: `CompanyOverview` with `{'symbol': 'TSLA'}`


{'Symbol': 'TSLA', 'AssetType': 'Common Stock', 'Name': 'Tesla Inc', 'Description': "Tesla, Inc. is an American electric vehicle and clean energy company based in Palo Alto, California. Tesla's current products include electric cars, battery energy storage from home to grid-scale, solar panels and solar roof tiles, as well as other related products and services. In 2020, Tesla had the highest sales in the plug-in and battery electric passenger car segments, capturing 16% of the plug-in market (which includes plug-in hybrids) and 23% of the battery-electric (purely electric) market. Through its subsidiary Tesla Energy, the company develops and is a major installer of solar photovoltaic energy generation systems in the United States. Tesla Energy is also one of the largest global suppliers of battery energy storage systems, with 3 GWh of battery storage supplied in 2020.", 'CIK': '1318605', 'Exchange': 'NASDAQ', 'Currency': 'USD', 

/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()



Invoking: `CompanyIncomeStatement` with `{'symbol': 'TSLA'}`


[{'fiscalDateEnding': '2024-12-31', 'reportedCurrency': 'USD', 'grossProfit': '17450000000', 'totalRevenue': '97690000000', 'costOfRevenue': '80240000000', 'costofGoodsAndServicesSold': '80240000000', 'operatingIncome': '7076000000', 'sellingGeneralAndAdministrative': '5150000000', 'researchAndDevelopment': '4540000000', 'operatingExpenses': '10374000000', 'investmentIncomeNet': 'None', 'netInterestIncome': '1219000000', 'interestIncome': '1569000000', 'interestExpense': '350000000', 'nonInterestIncome': 'None', 'otherNonOperatingIncome': 'None', 'depreciation': 'None', 'depreciationAndAmortization': '5368000000', 'incomeBeforeTax': '8990000000', 'incomeTaxExpense': '1837000000', 'interestAndDebtExpense': 'None', 'netIncomeFromContinuingOperations': '7153000000', 'comprehensiveIncomeNetOfTax': 'None', 'ebit': '9340000000', 'ebitda': '14708000000', 'netIncome': '7130000000'}, {'fiscalDateEnding': '2023-12-31', 'reportedCurr

/Users/yongjae/Documents/WEB-APP-Projects/FullStack-GPT/env/lib/python3.11/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


{'input': "Give me financial information on Tesla's stock, considering its financials, income statements help me analyze if it's a potential good investment.",
 'output': "Here is the financial information for Tesla Inc:\n\n- **Company Overview:**\n  - Name: Tesla Inc\n  - Description: Tesla, Inc. is an American electric vehicle and clean energy company based in Palo Alto, California. Tesla's current products include electric cars, battery energy storage from home to grid-scale, solar panels and solar roof tiles, as well as other related products and services. In 2020, Tesla had the highest sales in the plug-in and battery electric passenger car segments, capturing 16% of the plug-in market (which includes plug-in hybrids) and 23% of the battery-electric (purely electric) market. Through its subsidiary Tesla Energy, the company develops and is a major installer of solar photovoltaic energy generation systems in the United States. Tesla Energy is also one of the largest global suppliers